In [1]:
# HTML形式における文字幅は、半角1,全角2,と明確に決まっているわけではなく、文字の種類に応じて大きく異なる。　例えば、同じ半角アルファベットでも最大幅の'm'が18pxであるとき、最小幅の'j'はわずか5pxしかなかったりする。
# この文字幅の違いは、ルビサイズを決めるときに大きな問題になる。
# そこで、最も一般的に用いられる'Arial形式,16px'の際の文字幅をJSON形式で保存しておき、おおよその文字幅の基準として用いることとした。

import tkinter as tk
from tkinter import font
import json

def measure_bmp(family="Arial", size=16):
    """
    指定フォント (family, size) で、BMP (U+0000～U+FFFF) の全文字を測定し、
    {文字: 幅(px)} の辞書を返す。
    
    - サロゲートペア領域 (U+D800～U+DFFF) は単体文字として扱えないのでスキップ。
    - フォールバックにより、実際には Arial で描画されない文字もある可能性がある点に注意。
    """
    root = tk.Tk()
    root.withdraw()
    tk_font = font.Font(family=family, size=size)

    char_widths = {}
    count_skipped = 0  # サロゲートペア領域などをスキップした数

    # BMP の範囲
    start_cp = 0x0000
    end_cp   = 0xFFFF

    total_cp = end_cp - start_cp + 1

    print(f"BMP 計測開始: 0x{start_cp:04X} ～ 0x{end_cp:04X} (合計 {total_cp} コードポイント)")
    print("サロゲートペア領域 (0xD800～0xDFFF) はスキップします。")

    codepoint_list = []
    for cp in range(start_cp, end_cp + 1):
        # サロゲートペア領域をスキップ
        if 0xD800 <= cp <= 0xDFFF:
            count_skipped += 1
            continue
        codepoint_list.append(cp)

    print(f"実際に計測するコードポイント数: {len(codepoint_list)} (スキップ: {count_skipped})")

    # 計測ループ
    for i, cp in enumerate(codepoint_list, start=1):
        ch = chr(cp)
        try:
            w = tk_font.measure(ch)
            char_widths[ch] = w
        except:
            # 稀に発生する例外は無視
            pass

        # 進捗をある程度表示 (1万文字ごとなど、適宜調整)
        if i % 1000 == 0:
            print(f" - {i}/{len(codepoint_list)} 文字を計測完了 ...")

    root.destroy()
    return char_widths

def main():
    print("Measuring all BMP characters (excluding surrogate pair range) with Arial 16px...")
    widths_dict = measure_bmp(family="Arial", size=16)

    output_file = "Unicode_BMP全范围文字幅(宽)_Arial16.json"
    print(f"\n出力ファイル: {output_file} に保存します。")

    with open(output_file, "w", encoding="utf-8") as fp:
        json.dump(widths_dict, fp, ensure_ascii=False, indent=2)

    print(f"完了しました。 {len(widths_dict)} 文字の幅を保存しました。")

if __name__ == "__main__":
    main()


Measuring all BMP characters (excluding surrogate pair range) with Arial 16px...
BMP 計測開始: 0x0000 ～ 0xFFFF (合計 65536 コードポイント)
サロゲートペア領域 (0xD800～0xDFFF) はスキップします。
実際に計測するコードポイント数: 63488 (スキップ: 2048)
 - 1000/63488 文字を計測完了 ...
 - 2000/63488 文字を計測完了 ...
 - 3000/63488 文字を計測完了 ...
 - 4000/63488 文字を計測完了 ...
 - 5000/63488 文字を計測完了 ...
 - 6000/63488 文字を計測完了 ...
 - 7000/63488 文字を計測完了 ...
 - 8000/63488 文字を計測完了 ...
 - 9000/63488 文字を計測完了 ...
 - 10000/63488 文字を計測完了 ...
 - 11000/63488 文字を計測完了 ...
 - 12000/63488 文字を計測完了 ...
 - 13000/63488 文字を計測完了 ...
 - 14000/63488 文字を計測完了 ...
 - 15000/63488 文字を計測完了 ...
 - 16000/63488 文字を計測完了 ...
 - 17000/63488 文字を計測完了 ...
 - 18000/63488 文字を計測完了 ...
 - 19000/63488 文字を計測完了 ...
 - 20000/63488 文字を計測完了 ...
 - 21000/63488 文字を計測完了 ...
 - 22000/63488 文字を計測完了 ...
 - 23000/63488 文字を計測完了 ...
 - 24000/63488 文字を計測完了 ...
 - 25000/63488 文字を計測完了 ...
 - 26000/63488 文字を計測完了 ...
 - 27000/63488 文字を計測完了 ...
 - 28000/63488 文字を計測完了 ...
 - 29000/63488 文字を計測完了 ...
 - 30000/63488 文字を計測完了 ...
 

In [2]:
import json

# 事前に作成した Unicode_BMP全范围文字幅(宽)_Arial16.json ファイルを読み込み
with open("Unicode_BMP全范围文字幅(宽)_Arial16.json", "r", encoding="utf-8") as fp:
    widths_dict = json.load(fp)

In [3]:
def measure_text_size_json(text, widths_dict):
    """
    JSONで読み込んだ  {文字: 幅(px)} の辞書 (widths_dict) を用いて、
    与えられた文字列 text の幅を合計して返す。
    """
    total_width = 0
    for ch in text:
        # JSONにない文字の場合は幅 0 とみなす（または別の処理）
        char_width = widths_dict.get(ch, 8)
        total_width += char_width
    return total_width

In [4]:
text = "练"
width_px = measure_text_size_json(text, widths_dict)
print(f"文字列: {text}")
print(f"幅: {width_px}px")

文字列: 练
幅: 21px
